# Chat History

Here we'll be interacting with a server that's exposing a chat bot with message history being persisted on the backend.

In [1]:
# load environment variables
import dotenv
import os
dotenv.load_dotenv()
# set environment variables (in case different)
os.environ['REDIS_OM_URL'] = "redis://127.0.0.1:8001/0"
from svaeva_redux.schemas.redis import UserModel


In [2]:
# get all UserModels that have recently interacted
group_id = "metamersion3-test0"
count = 0
platform_id = "telegram"

users = UserModel.find(
            (UserModel.group_id == group_id)
        ).all()
    

InvalidResponse: Protocol error, got "H" as reply type byte

In [109]:
# client = redis.Redis(host='localhost', port=6379, decode_responses=True)





def bejeweled_gifter(users, level: int):
     # get all conversations from those users
    for user in users:
        print(user)
        # conversation_id = 6070074875
        # key = f"message_store:{conversation_id}"
        # conversation = client.lrange(key, 0, -1)


# get all users in group_id that fulfill the condition 0

def conversation_image_gifting_condition_interactions(group_id: str, interaction_count: int) -> None:
    """Users who have not received a gift after n interactions

    Args:
        group_id (str): the group id of the users to include
        interaction_count (int): the number of interactions to include

    Returns:
        None
    
    """
    # find all users beloning in group_id with interaction_count = n
    # users = client.smembers(f"group:{group_id}:users")

    users = UserModel.find(
            (UserModel.group_id == group_id) & 
            (UserModel.interaction_count == interaction_count) &
            (UserModel.platform_id == "telegram")
        )

    if interaction_count == 0:
        system_prompt = "" # level 0
        bejeweled_gifter(users, level=0)

    elif interaction_count == 5:
        system_prompt = "" # level 1
        bejeweled_gifter(users, level=1)



conversation_image_gifting_condition_interactions(group_id="metamersion3", interaction_count=0)

version: 1.0
commit: commit
id: Eric
group_id: metamersion3
platform_id: ipython
interaction_count: 0
first_name: None
last_name: None
username: None
description: 
phone_number: None
email: None
register_method: None
country: None
age: None
gender: None
english_proficiency: None
date_created_timestamp: 1709322541.066058
date_updated_timestamp: 1709322541.066075
date_accessed_timestamp: 1709470125.07686
date_updated: 2024-03-01T19:49:01.066074
date_accessed: 2024-03-03 12:48:45.076860


In [110]:
import uuid
from langserve import RemoteRunnable

import os
import sys
sys.path.append("/Users/eric/Library/CloudStorage/Dropbox/git/github/svaeva_redux/svaeva_redux")

%env REDIS_OM_URL=redis://127.0.0.1:6379/0
conversation_id = str(uuid.uuid4())
user_id = "Eric"
conversation_id = "1234"
platform_id = "ipython"

print(os.getenv("REDIS_OM_URL"))

env: REDIS_OM_URL=redis://127.0.0.1:6379/0
redis://127.0.0.1:6379/0


In [111]:
from svaeva_redux.schemas.redis import UserModel

user = UserModel(id=user_id, group_id="metamersion3", platform_id=platform_id)
user.save()
chat = RemoteRunnable("http://0.0.0.0:8000/", cookies={"user_id": user_id})
# chat.invoke(input={"human_input": "my name is eugene. what is your name?"})
output = chat.invoke({"human_input": "Count how many interactions we've had"}, config={"configurable": {"user_id": user_id, "conversation_id": conversation_id, "platform_id": platform_id}})
print(output)

content="As an AI, I don't have the ability to keep track of our past interactions. I'm designed to respect user privacy and confidentiality. However, I'm here to help you with any questions or tasks you have now." id=None name=None


Let's create a prompt composed of a system message and a human message.

In [86]:
user.increment_interaction_count()
user.interaction_count


50

In [15]:
from redis_om import Migrator
# Before running queries, we need to run migrations to set up the
# indexes that Redis OM will use. You can also use the `migrate`
# CLI tool for this!

# redis_conn = redis.from_url(os.getenv("REDIS_OM_URL"))
# redis_conn = redis.from_url("redis://127.0.0.1:6379/0")
Migrator().run()

In [17]:
user.interaction_count

1

In [89]:
from redis_om import NotFoundError
# get all users in the group

try:
    user = UserModel.get("asdasdasd")
except NotFoundError:
    print("User not found")

User not found


In [88]:
from redis_om import NotFoundError
try:
    user = UserModel.get(user_id)
except NotFoundError:
    print("User not found")


# get all users with number of interactions greater than 1
users = UserModel.find(
        (UserModel.interaction_count >= 1) &
        (UserModel.group_id == "metamersion3")
    ).all()

for user in users:
    print(user.id, user.interaction_count)
    # get conversations 
    

print(users)

Eric 50
[UserModel(version='1.0', commit='commit', id='Eric', group_id='metamersion3', platform_id='ipython', interaction_count=50, first_name=None, last_name=None, username=None, description='', phone_number=None, email=None, register_method=None, country=None, age=None, gender=None, english_proficiency=None, date_created_timestamp=1709222046.008508, date_updated_timestamp=1709222287.259773, date_accessed_timestamp=1709222365.882055, date_updated='2024-02-29T15:58:07.259772', date_accessed=datetime.datetime(2024, 2, 29, 15, 59, 25, 882062))]


In [26]:
from svaeva_redux.langchain.redis import RedisChatMessageHistoryWindowed

key_prefix = "message_store:"
url = "redis://127.0.0.1"
chat_history_length = 10

def retrieve_redis_windowed_chat_history_as_text(session_id: str, url: str, key_prefix: str, chat_history_length: int = 30) -> str:
    """
    Retrieve the chat history from Redis and return it as a string formatted for ingestion


    Args:
        session_id (str): The session id.
        url (str): The url of the Redis server.
        key_prefix (str): The key prefix for the chat history.
        chat_history_length (int): The length of the chat history.

    Returns:
        str: The chat history as a string.
    """
    
    history = RedisChatMessageHistoryWindowed(session_id=session_id, url=url, key_prefix=key_prefix, chat_history_length=chat_history_length)
    conversation = ""
    for message in history.messages:
        if message.type == "human":
            text = "Human: " + message.content
        elif message.type == "ai":
            text = "AI: " + message.content
        conversation += text + "\n"

    return conversation


conversation = retrieve_redis_windowed_chat_history_as_text(
    session_id=conversation_id,
    url=url,
    key_prefix=key_prefix,
    chat_history_length=chat_history_length
)

In [27]:
conversation

''

In [134]:
from langchain_community.chat_message_histories import (
    ChatMessageHistory,
    RedisChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory
truncated_message_history = ChatMessageHistory()
key_prefix = "message_store:"
history = RedisChatMessageHistory(session_id=user_id, url="redis://127.0.0.1", key_prefix=key_prefix)
history.messages = truncated_message_history.messages

AttributeError: property 'messages' of 'RedisChatMessageHistory' object has no setter

In [124]:
import redis

redis_client = redis.StrictRedis(host="localhost", port=6379, db=0)
if not redis_client.exists(":svaeva_redux.schemas.conversation.ConversationModel:default_0"):
    print("Database initialized")

In [ ]:
class OpenAIConfig(BaseModel):
    api_key: Optional[str] = None
    engine: str = "gpt-4"
    temperature: float = Field(0.7, ge=0.0, le=1.0)
    max_tokens: int = Field(150, gt=0)
    model_token_limit: int = Field(8192, gt=0)
    top_p: float = Field(1.0, ge=0.0, le=1.0)
    frequency_penalty: float = Field(0.0, ge=0.0)
    presence_penalty: float = Field(0.0, ge=0.0)

    @validator('api_key')
    def api_key_must_be_provided(cls, v, values):
        if v is None:
            raise ValueError("API key must be provided.")
        return v

    @validator('engine')
    def engine_must_be_valid(cls, v):
        valid_engines = ["text-davinci-003", "text-davinci-004", "text-curie-003"]  # Add more engines as needed
        if v not in valid_engines:
            raise ValueError(f"Invalid engine '{v}'. Valid engines are: {', '.join(valid_engines)}")
        return v

    @validator('temperature', 'top_p', 'frequency_penalty', 'presence_penalty')
    def value_must_be_within_range(cls, v):
        if v < 0 or v > 1:
            raise ValueError("Value must be between 0 and 1.")
        return v

    @validator('max_tokens')
    def max_tokens_must_be_positive(cls, v):
        if v <= 0:
            raise ValueError("Max tokens must be a positive integer.")
        return v

In [69]:
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from numpy import clip
user_id = "Eric"
REDIS_URL = "redis://localhost:6379/0"
key_prefix = "message_store:"
history = RedisChatMessageHistory(session_id=user_id, url=REDIS_URL, key_prefix=key_prefix)
print(len(history.messages))

# from langchain_core.runnables.history import RunnableWithMessageHistory

6


In [60]:
from regex import B
from langchain_core.runnables.history import RunnableWithMessageHistory

# max sure max_length is even number
max_length = 2
if max_length % 2 != 0 and max_length > 0:
    max_length -= 1

message_history = ChatMessageHistory()
for message in history.messages:
    if len(message_history.messages) >= max_length:
        break
    # clipped_messages.append(message)
    message_history.add_message(message)

print(message_history.messages)
# message_history.aadd_messages(

[HumanMessage(content='my name is eugene. what is your name?'), AIMessage(content="Does it really matter? I mean, we're just talking here. No need for formalities.")]


In [39]:
chat.invoke({"human_input": "What's my name?"}, {"configurable": {"conversation_id": conversation_id, "platform_id": platform_id}})

AIMessage(content='Your name is Eugene. How may I assist you today, Eugene?')

Use different user but same conversation id

In [5]:
chat = RemoteRunnable("http://localhost:8000/", cookies={"user_id": "nuno"})

In [6]:
chat.invoke({"human_input": "what was my name?"}, {'configurable': { 'conversation_id': conversation_id }})

AIMessage(content="I apologize, but I don't have access to personal information like names.")

In [7]:
for chunk in chat.stream({'human_input': "Can you count till 3?"},  {'configurable': { 'conversation_id': conversation_id } }):
    print()
    print(chunk.content, end='', flush=True)



Of
 course
!
 Here
 you
 go
:


1


2


3


In [10]:
import base64
image_vlm = RemoteRunnable("http://localhost:8000/image_vlm")
# Read the audio file in binary mode
with open('metamersion.png', 'rb') as image_file:
    image_data = image_file.read()

# Encode the audio data in Base64
encoded_image_data = base64.b64encode(image_data).decode('utf-8')
# chat.invoke({"human_input": "What's my name?"}, {"configurable": {"conversation_id": conversation_id, "platform_id": platform_id}})
image_vlm.invoke({"image": encoded_image_data, "configurable": {"user_id": user_id, "conversation_id": conversation_id, "platform_id": platform_id}})


AIMessage(content="It sounds like you're describing an image that portrays a variety of people engaging in different activities, depicted in a minimalist and monochrome style. As an AI, I don't have the ability to see or perceive images, but I can understand and respond to your description. It seems like this image may evoke a sense of diversity, movement, and everyday life. If you have any specific questions or thoughts about the image, feel free to share, and I'll do my best to assist you further.")

In [9]:
import base64
audio_stt = RemoteRunnable("http://localhost:8000/audio_stt")
# Read the audio file in binary mode
with open('test.m4a', 'rb') as audio_file:
    audio_data = audio_file.read()

# Encode the audio data in Base64
encoded_audio_data = base64.b64encode(audio_data).decode('utf-8')
print(encoded_audio_data[0:10], encoded_audio_data[-10:])
# chat.invoke({"human_input": "What's my name?"}, {"configurable": {"conversation_id": conversation_id, "platform_id": platform_id}})
audio_stt.invoke({"audio": encoded_audio_data, "configurable": {"user_id": user_id, "conversation_id": conversation_id, "platform_id": platform_id}})


AAAAHGZ0eX BDNjUpKQ==


AIMessage(content='I can hear you loud and clear, Eugene! How can I assist you today?')

In [112]:
import base64

with open("sample.pdf", "rb") as f:
    data = f.read()

encoded_data = base64.b64encode(data).decode("utf-8")

audio_stt.invoke({"file": encoded_data})

FileNotFoundError: [Errno 2] No such file or directory: 'sample.pdf'

In [17]:
!cat chat_histories/eugene/cd8e5a55-0295-41cd-a885-775e0403fd25.json | jq .

[
  {
    "type": "human",
    "data": {
      "content": "my name is eugene. what is your name?",
      "additional_kwargs": {},
      "type": "human",
      "example": false
    }
  },
  {
    "type": "ai",
    "data": {
      "content": "Hello Eugene! I'm Bob, your virtual assistant. How can I assist you today?",
      "additional_kwargs": {},
      "type": "ai",
      "example": false
    }
  },
  {
    "type": "human",
    "data": {
      "content": "what was my name?",
      "additional_kwargs": {},
      "type": "human",
      "example": false
    }
  },
  {
    "type": "ai",
    "data": {
      "content": "Your name is Eugene. Is there something specific you would like assistance with, Eugene?",
      "additional_kwargs": {},
      "type": "ai",
      "example": false
    }
  }
]


In [18]:
!cat chat_histories/nuno/cd8e5a55-0295-41cd-a885-775e0403fd25.json | jq .

[
  {
    "type": "human",
    "data": {
      "content": "what was my name?",
      "additional_kwargs": {},
      "type": "human",
      "example": false
    }
  },
  {
    "type": "ai",
    "data": {
      "content": "I apologize, but I don't have access to personal information about users. As an AI assistant, I prioritize user privacy and data protection.",
      "additional_kwargs": {},
      "type": "ai",
      "example": false
    }
  },
  {
    "type": "human",
    "data": {
      "content": "Can you count till 10?",
      "additional_kwargs": {},
      "type": "human",
      "example": false
    }
  },
  {
    "type": "AIMessageChunk",
    "data": {
      "content": "Of course! Here you go:\n\n1, 2, 3, 4, 5, 6, 7, 8, 9, 10.",
      "additional_kwargs": {},
      "type": "AIMessageChunk",
      "example": false
    }
  }
]
